**Esse arquivo é ignorado pelo Git. O script somente deve ser usado para testar gráficos antes de inserir no arquivo indicadores.py**

In [199]:
import pandas as pd
import requests
import sidrapy
import numpy as np
import scripts_saneamento
import datetime as dt

In [214]:
ipca = scripts_saneamento.importa_ipca('ipca_202210SerieHist.xls')

In [215]:
ipca['Data'] = pd.to_datetime(ipca['Data'], format='%m/%Y')
ipca.set_index('Data', inplace=True)

In [202]:
['IPCA - Variação acumulada em 12 meses', 'IPCA - Variação acumulada no ano', 'IPCA - Variação mensal']

['IPCA - Variação acumulada em 12 meses',
 'IPCA - Variação acumulada no ano',
 'IPCA - Variação mensal']

In [216]:
ipca['index'] = ipca.index.year

In [217]:
ipca['IPCA - Variação mensal'] = ipca['Índice'].pct_change(periods=1)*100

In [219]:
ipca['IPCA - Variação acumulada em 12 meses'] = ipca['Índice'].pct_change(periods=12)*100

In [224]:
ipca["IPCA - Variação acumulada no ano"] = ipca.groupby([ipca.index.year])["IPCA - Variação mensal"].transform(sum)

In [207]:
for mes in ipca.index.month:
    ipca['IPCA - Variação acumulada no ano'] = ipca['IPCA - Variação mensal'].rolling(mes).sum()

In [208]:
6407.93 / 141.31	

45.34661382775458

In [225]:
ipca.tail()

,index,Índice,IPCA - Variação mensal,IPCA - Variação acumulada em 12 meses,IPCA - Variação acumulada no ano
Data,,,,,
2022-06-01,2022,6455.85,0.67,11.887,4.63
2022-07-01,2022,6411.95,-0.68,10.069,4.63
2022-08-01,2022,6388.87,-0.36,8.727,4.63
2022-09-01,2022,6370.34,-0.29,7.169,4.63
2022-10-01,2022,6407.93,0.59,6.470,4.63


In [210]:
data_atual = dt.datetime.now() # captura data para determinar período na consulta da API
ipca_brasil = requests.get(f'https://apisidra.ibge.gov.br/values/t/7060/n1/all/v/63,69,2265/p/202101-{data_atual.year}{data_atual.month-1}/c315/7169/d/v63%202')
ipca_brasil = ipca_brasil.json()
ipca_brasil = pd.DataFrame(ipca_brasil)

## Tratamento dos dados do IPCA (Variação Mensal, Acumulado no Ano e Acumulado em 12 Meses)

ipca_brasil.columns = ipca_brasil.iloc[0] 
ipca_brasil = ipca_brasil[1:]
ipca_brasil['Valor'] = ipca_brasil['Valor'].astype(float)
ipca_brasil['Mês (Código)'] = pd.to_datetime(ipca_brasil['Mês (Código)'], format='%Y%m')

ipca_brasil = pd.crosstab(ipca_brasil['Mês (Código)'], ipca_brasil['Variável'], ipca_brasil['Valor'], aggfunc='sum')

In [211]:
ipca_brasil.columns

Index(['IPCA - Variação acumulada em 12 meses', 'IPCA - Variação acumulada no ano', 'IPCA - Variação mensal'], dtype='object', name='Variável')